# Run the final state algorithm on a ROOT HSDATA file

In [ ]:
gROOT->ProcessLine(".x LoadK2.C+");

First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.
Here for the K" g8 final state I want to only use particle ID for the tagger photons, named Beam and I want to loop over all Beam candidates. => fs(PID="Beam", INCLUSIVE="Beam") . All other particles are Ided by charge only and I demand an exclusive event for these.

In [ ]:
 K2 fs("Beam","Beam");
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

Create an output tree. This is defined by the TreeDataK2 class. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We must call fs.EndAndWrite() when we are ready to save and delete the tree save the tree. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [ ]:
fs.CreateFinalTree("tt","testcuts.root");

Here we add the cuts for each particle. This is done via a HS::Cuts object which gets propogated to each particle in each topology. To specify any type of cut behaviour you need to generate your own Cuts Method. Here we use a userdefined delta time cut made via [DeltaTimePCut](3b_Make_DeltaTimePCut.ipynb) .

In [ ]:
auto cutsman=make_shared<ParticleCutsManager>();
auto beamCut=make_shared<DeltaTimePCut>(7,3); //7ns delta time cut,  3GeV momentum 
auto otherCut=make_shared<DeltaTimePCut>(1,3); //10ns cut for all other particles
cutsman->AddParticleCut("Beam",beamCut); //assign to manager
cutsman->SetDefaultCut(otherCut);        //assign to manager
cutsman->ConfigureCuts(&fs);    //propogate the cuts to topologies in fs

Now register the cuts manager as a post topology function action. This will be applied after the Topo_0 etc functions when start times etc have been determined. 

In [ ]:
fs.RegisterPostTopoAction(cutsman);

Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager class.

In [ ]:
 //create datamanager
  auto dm=std::make_shared<DataManager>();

  //And make a chain of data files
  TChain chain("HSParticles");
  chain.Add("/home/dglazier/fastdata/g8/flist_048200.dat.root");
  dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);

Analyse all the events. Not you may prefer just looping over the data manager directly via while(dm->ReadEvent()) fs->Process();
You can also supply a number of events to analyse e.g. fs.ProcessData(100);

In [ ]:
cout<<"Number of Events to process "<<chain.GetEntries()<<endl;

fs.ProcessData(1E6); //No number give, analyse all events in chain
cout<<"Done "<<dm->GetEntry()<< " events "<<endl;

And remember to save the tree!

In [ ]:
//delete output;

In [ ]:
%jsroot on

In [ ]:
TCanvas c2;
fs.FinalTree()->Draw("MissMass>>(100,-1,2)","Topo==1");
fs.FinalTree()->Draw("MissMass","Topo==0","same");
fs.FinalTree()->Draw("MissMass","Topo==2","same");
fs.FinalTree()->Draw("MissMass","Topo==3","same");


In [ ]:
c2.Draw();

In [ ]:
fs.EndAndWrite()